In [1]:
#simple chain

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI()

parser = StrOutputParser()

chain = prompt | model | parser

result = chain.invoke({'topic':'cricket'})

print(result)

1. Cricket is one of the oldest team sports in the world, dating back to the 16th century.

2. The longest cricket match in history lasted for 14 days in 1939 and ended in a tie.

3. The highest individual score in international cricket was made by Brian Lara, who scored 400 not out in a test match in 2004.

4. The Ashes is one of the oldest and most prestigious cricket rivalries between England and Australia, dating back to 1882.

5. The term "hat-trick" in cricket refers to a bowler taking three wickets in three consecutive deliveries.


In [3]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [4]:
# sequential chain

In [5]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

model = ChatOpenAI()

parser = StrOutputParser()

chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({'topic': 'Unemployment in India'})

print(result)

chain.get_graph().print_ascii()

1. The unemployment rate in India was 7.2% in 2019-2020, affecting around 30 million people.
2. Lack of job creation in sectors like agriculture and manufacturing contributes to the unemployment issue.
3. Informal employment without job security and benefits is a common problem in India.
4. There is a mismatch between job seekers and job opportunities due to inadequacies in the educational system.
5. The COVID-19 pandemic has worsened the unemployment situation in India, particularly in industries like hospitality and tourism.
     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *      

In [6]:
#parallel chain

In [7]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel

load_dotenv()

model1 = ChatOpenAI()

model2  = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text':text})

print(result)

chain.get_graph().print_ascii()


**SVMs**

- SVMs are supervised learning methods for classification, regression, and outliers detection.
- Advantages of SVMs include effectiveness in high dimensional spaces, still effective when dimensions are greater than samples, memory efficiency, and versatility with Kernel functions.
- Disadvantages of SVMs include potential overfitting with a large number of features, lack of direct probability estimates, and costly probability calculation.
- SVMs in scikit-learn support dense and sparse sample vectors as input, with optimal performance using specific data types and format.

**Quiz**

1. **Question:** What are SVMs used for?
   **Answer:** Classification, regression, and outliers detection.

2. **Question:** What is a benefit of using SVMs in high dimensional spaces?
   **Answer:** They are effective in high dimensional spaces.

3. **Question:** What are support vectors?
   **Answer:** A subset of training points used in the decision function.

4. **Question:** What is a disadv

In [8]:
# conditional chain

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

model = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

chain.get_graph().print_ascii()

Okay, to provide the best response, I need a little more context!  Could you tell me:

1.  **What was the feedback ABOUT?** (e.g., a product, a service, a presentation, a piece of writing, a customer service interaction, a suggestion, etc.)
2.  **What was the gist of the feedback?** (e.g., "The product is amazing!", "The service was really helpful!", "The presentation was engaging!", "Your writing is clear and concise.")

**In the meantime, here are a few general responses that you can adapt:**

**General Options:**

*   "Thank you so much for your positive feedback! I'm so glad to hear that [it/you] had a good experience."
*   "I really appreciate you taking the time to share your thoughts. I'm happy to know that [the product/service/etc.] met your expectations."
*   "Thank you! It means a lot to me/us to know that you're happy with [the product/service/etc.]."
*   "We're thrilled to hear you enjoyed [the product/service/etc.]! Thanks for your support."

**More Specific Options (Depen